In [1]:
import numpy as np

In [ ]:
class KNearestNeighbor:
    '''
    A toy KNN classifier.

    k and data are all passed at initialization and are private attributes.
    '''
    def __init__(self, k:int, data:np.ndarray) -> None:
        self._k = k
        self._data = data
    
    def predict(self, x:np.ndarray) -> np.ndarray:
        pass

    

How to implement a getter for `_k` and `_data`?

Is it safe to create something like

```python
def get_k(self):
    return self._k
```

and 

```python
def get_data(self):
    return self._data
```

?

Let's try

In [8]:
class KNearestNeighbor:
    '''
    A toy KNN classifier.

    k and data are all passed at initialization and are private attributes.
    '''
    def __init__(self, k:int, data:np.ndarray) -> None:
        self._k = k
        self._data = data
    
    def predict(self, x:np.ndarray) -> np.ndarray:
        pass

    def get_k(self) -> int:
        return self._k
    
    def get_data(self) -> np.ndarray:
        return self._data


knn = KNearestNeighbor(3, np.array([[1,2,3], [4,5,6]]))

k = knn.get_k()
print(k)

data = knn.get_data()
print(data)

3
[[1 2 3]
 [4 5 6]]


So far so good, but is it really safe?
Since `_k` and `_data` are private, I would not want to modify it via the getter.

Let's try to modify `k` and `data` and see what happens

In [9]:
k = 5
data = data + 1

What happens inside `knn`?

In [10]:
print(knn.get_k())
print(knn.get_data())

3
[[1 2 3]
 [4 5 6]]


Notice that this behavior doesn't happen with `data` because it is a `numpy` array.
With other types, such as `list`, a reference to the list is returned by the getter, so it is possible to modify the list and reflect the changes in the object.

In [11]:
class KNearestNeighbor:
    '''
    A toy KNN classifier.

    k and data are all passed at initialization and are private attributes.
    '''
    def __init__(self, k:int, data:np.ndarray) -> None:
        self._k = k
        self._data = data
        self._internal_list = [1,2,3]
    
    def predict(self, x:np.ndarray) -> np.ndarray:
        pass

    def get_k(self) -> int:
        return self._k
    
    def get_data(self) -> np.ndarray:
        return self._data

    def get_list(self) -> list:
        return self._internal_list

In [14]:
knn2 = KNearestNeighbor(3, np.array([[1,2,3], [4,5,6]]))
list_ = knn2.get_list()
print(list_)
list_.append(4)
print(list_)
print(knn2.get_list())

[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3, 4]


`list_` and `knn.get_list()` reference the same object, so this is not the proper way to program the getter.

We need to return a **copy** of the list.
This can be done by using the `deepcopy` function from the `copy` module, which enables us to create a full copy of any Python object.

In [15]:
from copy import deepcopy

class KNearestNeighbor:
    '''
    A toy KNN classifier.

    k and data are all passed at initialization and are private attributes.
    '''
    def __init__(self, k:int, data:np.ndarray) -> None:
        self._k = k
        self._data = data
        self._internal_list = [1,2,3]
    
    def predict(self, x:np.ndarray) -> np.ndarray:
        pass

    def get_k(self) -> int:
        return self._k
    
    def get_data(self) -> np.ndarray:
        return self._data

    def get_list(self) -> list:
        return deepcopy(self._internal_list)

Now we can see what happens by running the code again

In [16]:
knn3 = KNearestNeighbor(3, np.array([[1,2,3], [4,5,6]]))
list_ = knn3.get_list()
print(list_)
list_.append(4)
print(list_)
print(knn3.get_list())

[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3]


Now `list_` and `knn3._internal_list` are two distinct objects, so we can modify `list_` without affecting `knn3._internal_list`.

If we want to allow the users to safely modify some private attributes, we can create a setter implementing a specific logic for the attribute.
For instance, we know that `k` in kNN should be larger than 0.

In [17]:
class KNearestNeighbor:
    '''
    A toy KNN classifier.

    k and data are all passed at initialization and are private attributes.
    '''
    def __init__(self, k:int, data:np.ndarray) -> None:
        self.set_k(k)
        self._data = data
        self._internal_list = [1,2,3]
    
    def predict(self, x:np.ndarray) -> np.ndarray:
        pass

    def get_k(self) -> int:
        return self._k
    
    def set_k(self, k:int) -> None:
        if k < 1:
            raise ValueError("k must be greater than 0")
        self._k = k
    
    def get_data(self) -> np.ndarray:
        return self._data

    def get_list(self) -> list:
        return deepcopy(self._internal_list)


Notice how we have changed the `__init__` part for setting k, so that we can re-apply the logic of the setter (checking for positive values of `k`) also in the constructor.

Let's see it in action:

In [18]:
knn5 = KNearestNeighbor(3, np.array([[1,2,3], [4,5,6]]))
print(knn5.get_k())
knn5.set_k(12)
print(knn5.get_k())
knn5.set_k(0)

3
12


ValueError: k must be greater than 0